# UpSet.js Jupyter Widget - Venn Diagram

UpSet.js has a basic support for classical Venn diagrams for two or three sets.

In [1]:
from ipywidgets import interact
from upsetjs_jupyter_widget import UpSetJSVennDiagramWidget
import pandas as pd

This wrapper is implemented in Python 3 with mypy typings and generics. The generic type `T` of the `UpSetJSVennDiagramWidget` is type of element that we wanna handle. In the following example we handle `str` elements.

In [2]:
w = UpSetJSVennDiagramWidget[str]()

## Basic User Interface

**Note**: The input data will be described in more detail in the next section

In [3]:
dict_input = dict(one = ['a', 'b', 'c', 'e', 'g', 'h', 'k', 'l', 'm'], two = ['a', 'b', 'd', 'e', 'j'], three = ['a', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm'])
w.from_dict(dict_input)

UpSetJSVennDiagramWidget(value=None, combinations=[UpSetSetDistinctIntersection(name=one, sets={'one'}, elems=…

## Input Formats

In the current version the UpSet.js wrapper supports two input data formats: `dictionary` and through a Pandas `dataframe`.

### Dictionary Input

The first format is a dictionary of type `Dict[str, List[T]]`, `T` refers again to the elements type, in this case it is a list of `str`. The key of the dictionary entry is the set name while the value is the list of elements this set has.

In [4]:
w.from_dict(dict(one = ['a', 'b', 'c', 'e', 'g', 'h', 'k', 'l', 'm'], two = ['a', 'b', 'd', 'e', 'j'], three = ['a', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm']))

UpSetJSVennDiagramWidget(value=None, combinations=[UpSetSetDistinctIntersection(name=one, sets={'one'}, elems=…

### Data Frame Input

The second format is a a binary/boolean data frame. The index column contains the list of elements. Each regular color represents a sets with boolean values (e.g., 0 and 1) whether the row represented by the index value is part of the set or not.

The following data frame defines the same set structure as the dictionary format before.

In [5]:
df = pd.DataFrame(dict(
    one=[1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1], 
    two=[1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0], 
    three=[1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1]
), index=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'])
w.from_dataframe(df)

UpSetJSVennDiagramWidget(value=None, combinations=[UpSetSetDistinctIntersection(name=one, sets={'one'}, elems=…

## Basic Attrributes

`.elems` returns the list of extracted elements

In [6]:
w.elems

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm']

`.sets` returns the list of extracted sets as `UpSetSet` objects

In [7]:
w.sets

[UpSetSet(name=one, elems={'e', 'k', 'b', 'g', 'l', 'a', 'm', 'c', 'h'}),
 UpSetSet(name=three, elems={'e', 'j', 'g', 'l', 'a', 'i', 'm', 'f', 'h'}),
 UpSetSet(name=two, elems={'a', 'e', 'd', 'j', 'b'})]

Similariy, `.combinations` returns the list of set intersections that are visualized as `UpSetIntersection` objects. 

**Note**: the attribute is called `.combinations` instead of `.intersections` since one customized the generation of the set combinations that are visualized. E.g., one can also generate set unions.

In [8]:
w.combinations

[UpSetSetDistinctIntersection(name=one, sets={'one'}, elems={'c', 'k'}),
 UpSetSetDistinctIntersection(name=three, sets={'three'}, elems={'i', 'f'}),
 UpSetSetDistinctIntersection(name=two, sets={'two'}, elems={'d'}),
 UpSetSetDistinctIntersection(name=(one ∩ three), sets={'three', 'one'}, elems={'m', 'g', 'l', 'h'}),
 UpSetSetDistinctIntersection(name=(one ∩ two), sets={'two', 'one'}, elems={'b'}),
 UpSetSetDistinctIntersection(name=(three ∩ two), sets={'three', 'two'}, elems={'j'})]

## Interaction

UpSet.js allows three intersection modes settable via `.mode`

- `'hover'` (default) when the user hovers over a set or set intersection it will be highlighted. This is the default mode
- `'click'` when the user clicks on a set or a set intersection, the selection will be updated
- `'contextMenu'` when the user right clicks on a set or a set intersection, the selection will be updated
- `'static'` disables interactivity

In [9]:
w.mode = 'click'
w

UpSetJSVennDiagramWidget(value=None, combinations=[UpSetSetDistinctIntersection(name=one, sets={'one'}, elems=…

with `.selection` one manually sets the selection that is currently highlighted. Manually setting the selection is only useful in `click` and `static` modes.

In [10]:
w.selection = w.sets[0]
w

UpSetJSVennDiagramWidget(value={'type': 'set', 'name': 'one'}, combinations=[UpSetSetDistinctIntersection(name…

The current selection is synced with the server. It is designed to work with the `interact` of the `ipywidgets` package. In the following example the current selected set will be automatically written below the chart and updated interactivly.

In [11]:
w.mode = 'hover'
def selection_changed(s):
    return s # s["name"] if s else None
interact(selection_changed, s=w)

interactive(children=(UpSetJSVennDiagramWidget(value={'type': 'set', 'name': 'one'}, combinations=[UpSetSetDis…

<function __main__.selection_changed(s)>

## Queries

besides the selection UpSet.js supports defining queries. A query can be a list of elements or a set that should be highlighted. A query consists of a name, a color, and either the list of elements or the set (combination) to highlight.

In [12]:
wq = w.copy()
wq.mode = 'static'
wq.selection = None
wq.append_query('Q1', color='red', elems=['a', 'b', 'c'])
wq.append_query('Q1', color='blue', upset=wq.sets[1])
wq

UpSetJSVennDiagramWidget(value=None, combinations=[UpSetSetDistinctIntersection(name=one, sets={'one'}, elems=…

## Styling

### Theme

UpSet.js supports three themes: `light`, `dark`, and `vega`. The theme can be set by the `.theme` property

In [14]:
w_dark = w.copy()
w_dark.theme = 'dark'
w_dark

UpSetJSVennDiagramWidget(value={'type': 'set', 'name': 'one'}, combinations=[UpSetSetDistinctIntersection(name…

### Labels

In [15]:
w_label = w.copy()
w_label.title = 'Chart Title'
w_label.description = 'a long chart description'
w_label

UpSetJSVennDiagramWidget(value={'type': 'set', 'name': 'one'}, combinations=[UpSetSetDistinctIntersection(name…